In [1]:
heading = '''<div style="border-top: 5px solid #000;
border-bottom: 1px solid #000;
padding-top: 8px;
padding-bottom: 8px;
margin-bottom: 24px;
line-height: 1.4;" data-test="title">
    <h2 class="u-ma-0">{}</h2>
</div>'''

style = '''<head><style>
    li.app-article-list-row__item{display:flex;border:solid}
    div.c-card__layout.u-full-height {
      display: flex;
      flex-direction: row-reverse;
    }
    div.c-card__section.c-meta {flex: 0 0 230px;}
  </style></head>'''

import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from datetime import datetime
def write_html(date = datetime.today()):
    class Article:
        def __init__(self,html:Tag) -> None:
            name_href = html.find("a",{"class": "c-card__link u-link-inherit"})
            self.name = name_href.text if name_href != None else ""
            self.url = "https://www.nature.com"+ name_href.attrs["href"] if name_href != None else ""
            self.date = datetime.strptime(html.find("time").attrs["datetime"],'%Y-%m-%d')
            self.magazine = html.find("span",{"class": "c-meta__type"}).text
            self.html = html
            pass

    site_indeces = "https://www.nature.com/siteindex"
    site_indeces_page = requests.get(site_indeces)

    soup = BeautifulSoup(site_indeces_page.content, 'html.parser')
    journals = soup.select("#back-to-top > div > div.grid.grid-8.mb20.mq640-grid-12.mq640-last > div")
    all_magazines = [r"https://www.nature.com"+journal.attrs["href"]+r"research-articles" for journal in journals[0].find_all('a')]
    articles_by_mags = {}
    len_maga = len(all_magazines)
    failed = []
    for num,magazine in enumerate(all_magazines):
        try:
            mag_soup = BeautifulSoup(requests.get(magazine).content, 'html.parser')
            articles_page = mag_soup.find_all("li",{"class": "app-article-list-row__item"})
            articles = [Article(article) for article in articles_page]
            articles_by_mags.update({magazine.split("/")[-2]: articles})
            print(f"Scanned {num+1} out of {len_maga} magazines",end="\r")
        except:
            print(f"Error with {magazine}")
            failed.append(magazine)
            pass



    with open("index.html","w+",encoding="utf-8") as file:
        file.write("")
    with open("index.html","a+",encoding="utf-8") as file:
        file.write("<html>")
        file.write(style)
        file.write("<body>")
        len_mag = len(articles_by_mags.items())
        for num,(mag, articles) in enumerate(articles_by_mags.items()):
            file.write(heading.format(mag))
            for num,article in enumerate(articles):
                if article.date.date() >= date.date():
                    file.write(str(article.html))
                    file.write(f"<a href='{article.url}'>Link</a>")
                    file.write("<br>")
                else:
                    file.write("<br>")
                    break
                file.write("<br>")
            file.write("<br>")
            print(f"Written {num} out of {len_mag}",end="\r")
        file.write("<br>")
        file.write("</body></html>")
write_html()

In [36]:
requests.get("https://www.nature.com/npjpcrm/research-articles")

<Response [200]>